In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM, TimeDistributed
from keras.layers import Concatenate, Flatten
from keras.layers import GRU, Conv2D, MaxPooling2D
from keras.layers import Input, Reshape, Dot
from keras.models import Model
from keras.optimizers import Adam
from keras.optimizers import RMSprop
# from keras.utils.vis_utils import plot_model
import keras
import keras.backend as K
from data_handle import *
from gensim_wrapper import *
from utils import *

In [ ]:
def conv_model(n_input, n_output, n_enc_units, n_dec_units):
    root_word_input = Input(shape=(13, 309, 1), name="root_word_input")
    
    x = Conv2D(16, (3, 3), padding='same', activation='relu')(root_word_input)
    x = MaxPooling2D(2, 2)(x)
    x = Conv2D(8, (3, 3), padding='same', activation='relu')(x)
    x = MaxPooling2D(2, 2)(x)
    
    x = Flatten()(x)

    state_out = Dense(n_dec_units, activation='tanh')(x)
    state_h = Dense(n_dec_units, activation='relu')(state_out)
    
    decoder_inputs = Input(shape=(None, 309), name="target_word_input")
    decoder_gru = GRU(n_dec_units, return_sequences=True, return_state=True, name="decoder_gru")
    decoder_outputs, _= decoder_gru(decoder_inputs, initial_state=state_h)
    
    decoder_dense = Dense(309, activation='softmax', name="train_output")
    decoder_outputs = decoder_dense(decoder_outputs)
    
    model = Model([root_word_input, decoder_inputs], decoder_outputs)
    encoder_model = Model(root_word_input, state_out)
    
    decoder_state_input_h = Input(shape=(n_dec_units,))
    decoder_outputs, state_h= decoder_gru(decoder_inputs, initial_state=decoder_state_input_h)

    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = Model([decoder_inputs, decoder_state_input_h], [decoder_outputs, state_h])

    return model, encoder_model, decoder_model


In [ ]:
def embedding_model(input_size, output_size, embed_size):
    context_word = Input(shape=(input_size,), name="context_word")
    x = Dense(256, activation='relu')(context_word)
    embeding = Dense(embed_size, activation='tanh')(x)
    target_word = Dense(output_size, activation='relu')(embeding)
    model = Model(context_word, target_word)
    em_model = Model(context_word, embeding)
    return model, em_model

In [ ]:
def cosine_loss(yTrue, yPred):
    loss = K.sum(K.square(yTrue - yPred))
    return loss

In [ ]:
def embedding_model2(input_size, output_size, embed_size):
    context_word = Input(shape=(input_size,), name="context_word")
    target_word = Input(shape=(input_size,), name="target_word")
    
    layer1 = Dense(200, activation='tanh')
    layer2 = Dense(200, activation='tanh')
    
    x = layer1(context_word)
    y = layer1(target_word)
#     y = layer2(y)
    cosine_sim = Dot(normalize=True, axes=1)([x, y])
#     z = Concatenate(axis=1)([x, y])
#     z = Dense(20, activation='tanh')(z)
    
#     output = Dense(1, activation='tanh')(z)
    model = Model([context_word, target_word], cosine_sim)
    
    con_model = Model(context_word, x)
    tar_model = Model(target_word, y)
    
    return model, con_model, tar_model

In [ ]:
def embedding_model3(input_size, output_size, embed_size):
    left_word = Input(shape=(input_size,), name="left_word")
    right_word = Input(shape=(input_size,), name="right_word")
    
    layer1 = Dense(128, activation='relu')
    layer2 = Dense(128, activation='linear')
    
    left = layer1(left_word)
    right = layer1(right_word)
    
    left = layer2(left)
    right = layer2(right)
    
    x = Concatenate(axis=1)([left, right])
    x = Dense(embed_size, activation='tanh')(x)
    
    model = Model([left_word, right_word], x)
    con_model = Model(left_word, left)
    tar_model = Model(right_word, right)
    
    return model, con_model, tar_model

In [ ]:
def evaluate(final_embedding, word2int, embed_size):
    gensim = GensimWrapper(embed_size, 0, log=False)
    gensim.set_embeddings(word2int, final_embedding)
    result = gensim.evaluate()
    for key in result:
        print("{0}: {1:.2f}%".format(key, result[key]), end=' ')
    print()

In [ ]:
def normalize(embeddings):
    norms = np.linalg.norm(embeddings, axis=1, keepdims=True)
    return embeddings / norms

In [ ]:
words = read_file()
vocab, word2int, int2word = build_vocab(words)
word2freq = get_frequency(words, word2int, int2word)
char2int, int2char, char2tup, tup2char, n_consonant, n_vowel = build_charset()
ns_unigrams = ns_sample(word2freq, word2int, int2word, .75)
n_chars = 11 + 2 
n_features = len(char2int)
batch_size = 250
embed_size = 128
skip_window = 3

In [ ]:
gen = generate_batch_image_v3(words, word2int, char2int, batch_size, skip_window)

In [ ]:
train, infenc, infdec = conv_model(13, 13, embed_size, embed_size)
train.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])

In [ ]:
n_batches = len(words) * 2 * skip_window  // batch_size
history = train.fit_generator(gen, steps_per_epoch=n_batches, epochs = 4)

In [ ]:
embeddings = np.ndarray((len(vocab), embed_size))
i = 0
buffer = []
buffer_size = 10000
for i_word in range(len(vocab)):
    word = int2word[i_word]
    buffer.append(word2vec(char2int, word, 13))
    if len(buffer) == buffer_size or len(vocab) - i_word < buffer_size:
        buffer_np = np.stack(buffer).reshape((-1, 13, 309, 1))
        result = infenc.predict(buffer_np)
        embeddings[i:i+len(buffer)] = result
        i += len(buffer)
        buffer = []
        if i % (4 *buffer_size) == 0:
            print("Predicting: {0:.2f}%".format((i * 100.0 / len(vocab))))
print("finished")

In [ ]:
embedding_normal = normalize(embeddings)

In [ ]:
evaluate(embedding_normal, embed_size=embed_size, word2int=word2int)

In [ ]:
# min = 1
# b = embedding_normal[0]
# for i in range(len(embedding_normal)):
#     a = embedding_normal[i]
#     d = a.dot(b)
#     if d < min:
#         print(d)
#         min = d
# embedding_normal[0]

In [ ]:
def get_batch_words(words, start, length):
    if start + length > len(words):
        end = start + length - len(words)
        return words[start:] + words[0:end], end
    else:
        end = start + length
        return words[start:end], end

# def get_context_words(words, start, length):
#     if start + length > len(words):
#         start = 0
#     end = start + length
#     return words[start:end], end


In [ ]:
def generate(data, embeds, word2int, int2word, unigrams, batch_size, skip_window):
    embed_szie = embeds.shape[1]
    assert batch_size % skip_window == 0
    ci = skip_window  # current_index
    batch_y = np.ones(shape=(batch_size, 1), dtype=np.float32)
    while True:
        batch_inputs = np.ndarray(shape=(batch_size, embed_size), dtype=np.float32)
        batch_labels = np.ndarray(shape=(batch_size, embed_size), dtype=np.float32)
        batch_index = 0
        shuffle_index = np.random.shuffle(np.arange(batch_size))
        for batch_index in range(0, batch_size, skip_window * 2):  # fill the batch inputs
            context = data[ci - skip_window:ci + skip_window + 1]
            # remove the target from context words
            target = context.pop(skip_window)
            # context = random.sample(context, skip_window * 2)
            word_index = 0
            for b in range(batch_index, batch_index + skip_window * 2):
                con_vec = embeds[word2int[context[word_index]]]
                target_vec = embeds[word2int[target]]
                batch_inputs[b] = con_vec
                batch_labels[b] = target_vec
                word_index += 1

            ci += 1
        if len(data) - ci - skip_window < batch_size:
            ci = skip_window
        for ri  in range(0, batch_size, 2):
            batch_labels[ri] = embeds[np.random.randint(len(embeds))]
            batch_y[ri][0] = batch_labels[ri].dot(batch_inputs[ri])
#         print(batch_labels.shape)
#         batch_labels = batch_labels[shuffle_index].reshape((-1, 128))
# #         print(batch_labels.shape)
#         batch_inputs = batch_inputs[shuffle_index].reshape((-1, 128))
#         batch_y = batch_y[shuffle_index].reshape((-1, 1))
        yield [batch_inputs, batch_labels], batch_y


In [ ]:
def generate2(data, embeds, word2int, batch_size, skip_window):
    embed_size = embeds.shape[1]
    assert batch_size % skip_window == 0
    ci = skip_window  # current_index
    input_width = embed_size * 2 * skip_window
    while True:
        batch_inputs = np.ndarray(shape=(batch_size, input_width), dtype=np.float32)
        batch_labels = np.ndarray(shape=(batch_size, embed_size), dtype=np.float32)
        batch_index = 0
        for batch_index in range(batch_size):  # fill the batch inputs
            context = data[ci - skip_window:ci + skip_window + 1]
            target = context.pop(skip_window)
#             print(context, target)
            context_vec = []
            target_vec = embeds[word2int[target]]
            for word in context:
                con_vec = embeds[word2int[word]]
                context_vec.append(con_vec)
            context_vec = np.hstack(context_vec)
#             batch_inputs[batch_index] = context_vec
#             batch_labels[batch_index] = target_vec
            
            ci += 1
        if len(data) - ci - skip_window < batch_size:
            ci = skip_window
        yield batch_inputs, batch_labels


In [ ]:
def generate3(data, embeds, word2int, batch_size, skip_window):
    embed_size = embeds.shape[1]
    ci = 0  # current_word_index
    input_width = embed_size * 2 * skip_window
    while True:
        batch_inputs_left = np.ndarray(shape=(batch_size, embed_size), dtype=np.float32)
        batch_inputs_right = np.ndarray(shape=(batch_size, embed_size), dtype=np.float32)
        batch_labels = np.ndarray(shape=(batch_size, embed_size), dtype=np.float32)
        batch_index = 0
        for batch_index in range(batch_size):  # fill the batch inputs
            context, ci = get_context_words(data, ci, 3)
            ci = ci - 2
            left_word_vec = embeds[word2int[context[0]]]
            target_vec = embeds[word2int[context[1]]]
            right_word_vec = embeds[word2int[context[2]]]
            batch_inputs_left[batch_index] = left_word_vec
            batch_inputs_right[batch_index] = right_word_vec
            batch_labels[batch_index] = target_vec

        yield [batch_inputs_left, batch_inputs_right], batch_labels


In [ ]:
gen3 = generate3(words, embedding_normal, word2int, batch_size, 3)

In [ ]:
window = 3
semantic_batch_size = 120
input_size = 128
# gg =  generate2(words, embeds_norm, word2int, batch_size=semantic_batch_size, skip_window=window) 

In [ ]:
train_model, con_model, tar_model = embedding_model3(input_size, 128, embed_size)
adam = keras.optimizers.Nadam(0.0001)
train_model.compile(optimizer=adam, loss="mse")
train_model.summary()

In [ ]:
n_batches = len(words) // semantic_batch_size
history = train_model.fit_generator(gen3, steps_per_epoch=n_batches, epochs = 2)

In [ ]:
g = generate(words, embedding_normal, word2int, int2word, ns_unigrams, batch_size=semantic_batch_size, skip_window=3)
[a, b], y = next(g)
print(y)

In [ ]:
input_size = 128#window * 2* embed_size
# em_train, em_out = embedding_model(input_size, 128, embed_size)
# adam = keras.optimizers.Nadam(lr=0.002)
# em_train.compile(optimizer=adam, loss='mean_squared_error')
# em_train.summary()

In [ ]:
train_model, con_model, tar_model = embedding_model2(input_size, 128, embed_size)
adam = keras.optimizers.SGD(0.001)
train_model.compile(optimizer=adam, loss="mse", metrics=['mse', 'acc'])
train_model.summary()

In [ ]:
n_batches = len(words) // semantic_batch_size
history = train_model.fit_generator(g, steps_per_epoch=n_batches, epochs = 2)

In [ ]:
context_vecs = []
for i_word in range(len(vocab)):
    word = int2word[i_word]
    context_vecs.append(embedding_normal[word2int[word]])
context_vecs = np.stack(context_vecs)
context_embed = con_model.predict(context_vecs)
# target_embed = tar_model.predict(context_vecs)
# em = context_embed + target_embed

In [ ]:
em_normal = normalize(context_embed)
evaluate(em_normal, word2int, embed_size=em_normal.shape[1])

In [ ]:
utils = Utils(word2int, em_normal)
v = -em_normal[word2int['ገንዘብ']] + em_normal[word2int['ብር']]
dots = em_normal.dot(v).flatten()
int2word[np.argmax(dots)]
utils.sorted_sim("ዶላር")
utils.sorted_sim("ብር")

In [ ]:
con_input = [
    embedding_normal[word2int['ነበር']].reshape((1, 128)),
    embedding_normal[word2int['ነው']].reshape((1, 128)),
]
vec = train_model.predict(con_input).flatten()
print(embedding_normal[word2int['ነበር']].dot(embedding_normal[word2int['ነው']]))
int2word[np.argmax(embedding_normal.dot(vec))]

In [ ]:
min = 1
b = em_normal[0]
for i in range(len(em_normal)):
    a = em_normal[i]
    d = a.dot(b)
    if d < min:
        print(d)
        min = d
# embedding_normal[0]

In [ ]:
# semantic = em_out.predict(embeds_norm)
# gensim = GensimWrapper(embed_size, 0, log=True)
# embeds = embeds.reshape((-1, 128))
# norms = np.linalg.norm(semantic, axis=1, keepdims=True)
# semantic_norm = semantic / norms
vecs = []
discovered = {}
for i in range(window, len(words) - window):
    context = words[i - window: i + window + 1]
    target = context.pop(window)
    if target not in discovered:
        discovered[target] = len(discovered)
        c_vec = []
        for cword in context:
            vec = embeds_norm[word2int[cword]]
            c_vec.append(vec)
        context_vec = np.hstack(c_vec)
        vecs.append(context_vec)
    if len(discovered) == len(vocab):
        print("discovered")
        break
    
semantic = np.stack(vecs).reshape(-1, input_size)
print(len(vecs), embeds_norm.shape)
assert semantic.shape[0] == embeds_norm.shape[0]
# semantic = em_out.predict(embeds_norm)
# gensim = GensimWrapper(embed_size, 0, log=True)
# embeds = embeds.reshape((-1, 128))
# norms = np.linalg.norm(semantic, axis=1, keepdims=True)
# semantic_norm = semantic / norms
